In [60]:
import pandas as pd
import matplotlib.pyplot as plt
import scipy.stats as st
from scipy.stats import linregress
import numpy as np

In [61]:
mouse_metadata=pd.read_csv("module_5_starter_code/Pymaceuticals/data/Mouse_metadata.csv")
mouse_metadata.sample(10)

,Mouse ID,Drug Regimen,Sex,Age_months,Weight (g)
192,s619,Stelasyn,Male,22,30
228,x226,Ceftamin,Male,23,28
57,a577,Infubinol,Female,6,25
36,v923,Capomulin,Female,19,21
29,c458,Ramicane,Female,23,20
16,t565,Capomulin,Female,20,17
189,s187,Propriva,Male,22,25
198,t994,Placebo,Male,14,30
168,p189,Ketapril,Male,8,28
225,w746,Propriva,Male,1,26


In [62]:
study_results=pd.read_csv("module_5_starter_code/Pymaceuticals/data/Study_results.csv")
study_results.sample(10)

,Mouse ID,Timepoint,Tumor Volume (mm3),Metastatic Sites
470,w167,5,48.786272,0
1870,g791,45,29.128472,1
1508,g296,35,60.384520,4
1239,w151,25,51.869059,2
728,h246,15,56.184327,1
1063,e662,20,44.001296,0
1498,q610,35,36.561652,2
6,p189,0,45.000000,0
1474,a963,35,56.813976,1
1733,w575,40,64.938406,3


In [63]:
combined_df=pd.merge(mouse_metadata,study_results,on="Mouse ID", how="outer")
# Checking the number of mice.
id_dataframe=combined_df["Mouse ID"].unique()
print(len(id_dataframe))

249


In [64]:
# Getting the duplicate mice by ID number that shows up for Mouse ID and Timepoint. 
duplicate=combined_df[combined_df.duplicated(['Mouse ID', 'Timepoint'])]
print(duplicate)

    Mouse ID Drug Regimen     Sex  Age_months  Weight (g)  Timepoint  \
909     g989     Propriva  Female          21          26          0   
911     g989     Propriva  Female          21          26          5   
913     g989     Propriva  Female          21          26         10   
915     g989     Propriva  Female          21          26         15   
917     g989     Propriva  Female          21          26         20   

     Tumor Volume (mm3)  Metastatic Sites  
909           45.000000                 0  
911           47.570392                 0  
913           49.880528                 0  
915           53.442020                 0  
917           54.657650                 1  


In [65]:
# Optional: Get all the data for the duplicate mouse ID. 
g989_mouth = combined_df.loc[combined_df["Mouse ID"] == "g989"]
g989_mouth 

,Mouse ID,Drug Regimen,Sex,Age_months,Weight (g),Timepoint,Tumor Volume (mm3),Metastatic Sites
908,g989,Propriva,Female,21,26,0,45.000000,0
909,g989,Propriva,Female,21,26,0,45.000000,0
910,g989,Propriva,Female,21,26,5,48.786801,0
911,g989,Propriva,Female,21,26,5,47.570392,0
912,g989,Propriva,Female,21,26,10,51.745156,0
913,g989,Propriva,Female,21,26,10,49.880528,0
914,g989,Propriva,Female,21,26,15,51.325852,1
915,g989,Propriva,Female,21,26,15,53.442020,0
916,g989,Propriva,Female,21,26,20,55.326122,1
917,g989,Propriva,Female,21,26,20,54.657650,1


In [77]:
# Create a clean DataFrame by dropping the duplicate mouse by its ID.
cleaned_df = combined_df.loc[combined_df["Mouse ID"] != "g989"]
cleaned_df

,Mouse ID,Drug Regimen,Sex,Age_months,Weight (g),Timepoint,Tumor Volume (mm3),Metastatic Sites
0,k403,Ramicane,Male,21,16,0,45.000000,0
1,k403,Ramicane,Male,21,16,5,38.825898,0
2,k403,Ramicane,Male,21,16,10,35.014271,1
3,k403,Ramicane,Male,21,16,15,34.223992,1
4,k403,Ramicane,Male,21,16,20,32.997729,1
...,...,...,...,...,...,...,...,...
1888,z969,Naftisol,Male,9,30,25,63.145652,2
1889,z969,Naftisol,Male,9,30,30,65.841013,3
1890,z969,Naftisol,Male,9,30,35,69.176246,4
1891,z969,Naftisol,Male,9,30,40,70.314904,4


In [78]:
id=cleaned_df["Mouse ID"].nunique()



248

Summary Statistics

In [88]:
# Generate a summary statistics table of mean, median, variance, standard deviation, and SEM of the tumor volume for each regimen

# Use groupby and summary statistical methods to calculate the following properties of each drug regimen: 
# mean, median, variance, standard deviation, and SEM of the tumor volume. 
# Assemble the resulting series into a single summary DataFrame.
mean_tumor_volume=cleaned_df.groupby(["Drug Regimen"])["Tumor Volume (mm3)"].mean()
median_tumor_volume=cleaned_df.groupby(["Drug Regimen"])["Tumor Volume (mm3)"].median()
variance_tumor_volume=cleaned_df.groupby(["Drug Regimen"])["Tumor Volume (mm3)"].var()
std_tumor_volume=cleaned_df.groupby(["Drug Regimen"])["Tumor Volume (mm3)"].std()
sterror_tumor_volume=cleaned_df.groupby(["Drug Regimen"])["Tumor Volume (mm3)"].sem()


summary_drugs={
    "Mean Tumor Volume":mean_tumor_volume,
    "Median Tumor Volume":median_tumor_volume,
    "Tumor Volume Variance":variance_tumor_volume,
    "Tumor Volume Std. Dev.":std_tumor_volume,
    "Tumor Volume Std. Err.":sterror_tumor_volume
}



summary_drugs = pd.DataFrame(summary_drugs)
summary_drugs.index.name = "Drug Regimen"

# Display results
summary_drugs

,Mean Tumor Volume,Median Tumor Volume,Tumor Volume Variance,Tumor Volume Std. Dev.,Tumor Volume Std. Err.
Drug Regimen,,,,,
Capomulin,40.675741,41.557809,24.947764,4.994774,0.329346
Ceftamin,52.591172,51.776157,39.290177,6.268188,0.469821
Infubinol,52.884795,51.820584,43.128684,6.567243,0.492236
Ketapril,55.235638,53.698743,68.553577,8.279709,0.603860
Naftisol,54.331565,52.509285,66.173479,8.134708,0.596466
Placebo,54.033581,52.288934,61.168083,7.821003,0.581331
Propriva,52.320930,50.446266,43.852013,6.622085,0.544332
Ramicane,40.216745,40.673236,23.486704,4.846308,0.320955
Stelasyn,54.233149,52.431737,59.450562,7.710419,0.573111


In [100]:
# Generate a summary statistics table of mean, median, variance, standard deviation, 
# and SEM of the tumor volume for each regimen

# Using the aggregation method, produce the same summary statistics in a single line.
summary_drug_al = cleaned_df.groupby(["Drug Regimen"])["Tumor Volume (mm3)"].describe(percentiles=None, include=[var,median,sem], exclude=[count,min,max])
summary_drug_al

NameError: name 'var' is not defined